In [1]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.inception_v3 import preprocess_input
import os
from ultralytics import YOLO


#import easyocr
#Load EasyOCR reader
#reader = easyocr.Reader(['en'])


# Load YOLOv8 model
model_path = "runs/detect/train4/weights/best.pt"  # Replace with the correct path to your YOLOv8 model
model = YOLO(model_path)

# List of categories
plat = ['AA', 'AB', 'AD', 'AE', 'AG', 'B', 'BE', 'BK', 'BM', 'D', 'DA', 'DH', 'DK', 'E', 'F', 'G', 'H', 'K', 'KB', 'KH', 'KT', 'KU', 'L', 'M', 'N', 'P', 'R', 'S', 'T', 'W']

# Load the trained classification model (InceptionV3 or similar)
classification_model = tf.keras.models.load_model("daerah.keras")

# Define the image size expected by the classification model (299x299)
img_height = 299
img_width = 299



In [3]:

# Function to preprocess the image specifically for FE-Schrift font
def preprocess_image(image):
    #grayscale_img = img.convert("L")
    
    if len(image.shape) == 3:  # Check if the image has three channels (color image)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image  # Already grayscale
        
    #denoised = cv2.bilateralFilter(gray, 11, 17, 17)
    #_, adaptive_thresh = cv2.threshold(denoised, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #if checkwhite(adaptive_thresh):
    #    adaptive_thresh = invert_colors(adaptive_thresh)

    #return adaptive_thresh
    return gray

# Load and preprocess the image for classification prediction
def load_image_for_prediction(img):
    img = cv2.resize(img, (img_width, img_height))  # Resize image to 299x299
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = preprocess_input(img)  # Preprocess for InceptionV3 or similar model
    return img

# Predict the class of the cropped plate image
def predict_image(img):
    img_array = load_image_for_prediction(img)
    predictions = classification_model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class[0]  # Return the predicted class index

# Process frame for license plate detection and classification
def process_frame(frame, outputpath, filename, pos):
    """Process frame for license plate detection and classification."""
    frame = cv2.resize(frame, (1024, 820))  # Resize the image for YOLO detection (if needed)

    # Detect plates using YOLOv8
    results = model(frame)

    i = 1
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy().astype(int)  # Get bounding boxes in xyxy format
        confidences = result.boxes.conf.cpu().numpy()  # Get confidence scores

        for box, conf in zip(boxes, confidences):
            if conf > 0.5:  # Only consider boxes with confidence > 0.5
                x1, y1, x2, y2 = box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                plate_image = frame[y1:y2, x1:x2]  # Crop the plate from the image
                plate_image = cv2.resize(plate_image, (400,200))

                # Resize the cropped plate image for classification
                plate_image1 = plate_image[10:128, 10:95]
                                
                # Use the classification model to predict the class of the plate
                plate_image1 = cv2.resize(plate_image1, (img_width, img_height))
                predicted_class_index = predict_image(plate_image1)
                plate_text1 = plat[predicted_class_index]
                print(" License: ", plate_text1) 

                # Optionally, draw the predicted class on the image
                cv2.putText(frame, plate_text1, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Save the processed plate image
                #cv2.imwrite(outputpath + "/" + filename + "-" + str(pos) + "-" + str(i) + "-L.png", plate_image)

                i += 1

    return frame

In [5]:
#gambar statis
# Paths
image_folder = "test"  # Folder containing images
output_folder = "testpredict"  # Folder to save output images


# Create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process all images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)        
        image = process_frame(image, output_folder, filename, 1)

        output_file = os.path.join(output_folder, f"result_{filename}")
        cv2.imwrite(output_file, image)

        #print(f"Processed: {filename}")
        #print("---")

print("Processing complete.")



0: 512x640 1 license, 358.2ms
Speed: 15.6ms preprocess, 358.2ms inference, 19.3ms postprocess per image at shape (1, 3, 512, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
 License:  H

0: 512x640 1 license, 157.2ms
Speed: 0.0ms preprocess, 157.2ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
 License:  H

0: 512x640 1 license, 173.0ms
Speed: 0.0ms preprocess, 173.0ms inference, 14.6ms postprocess per image at shape (1, 3, 512, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
 License:  H

0: 512x640 1 license, 204.4ms
Speed: 0.0ms preprocess, 204.4ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
 License:  H

0: 512x640 1 license, 204.5ms
Speed: 0.0ms preprocess, 204.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
 License:  H

0: 512x640 1 license, 267.5ms
Speed: 0.0ms preprocess, 267.5ms inference, 0.0ms postprocess pe

KeyboardInterrupt: 

In [9]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
outputpath = "testpredict"
filename = '20240911_074146'
out = cv2.VideoWriter(outputpath + "/" + filename + ".avi", fourcc, 20.0, (1024, 820))


x, y, w, h = 10, 600, 2500, 2500  # Adjust these values as neede

# Use OpenCV to capture video from a file or webcam
video_path = "test/"+filename + ".mp4"  # If using a video file
cap = cv2.VideoCapture(video_path)  # For a video file
# cap = cv2.VideoCapture(0)  # Uncomment this line to use the webcam instead

# Check if the video capture opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()
    
frame_count = 0
pos = 1
# Loop over the frames of the video
while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Could not read frame.")
        break

    #frame_count += 1
    #if frame_count % 3 != 0:  # Adjust this number to skip frames
    #    continue
    
    #croper
    frame = process_frame(frame, outputpath, filename, pos)
    out.write(frame)
    pos = pos + 1
    
    # Display the frame with bounding boxes
    cv2.imshow('YOLOv5 Object Detection', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
out.release()
cv2.destroyAllWindows()



0: 512x640 1 license, 189.0ms
Speed: 15.6ms preprocess, 189.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 license, 47.5ms
Speed: 0.0ms preprocess, 47.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 license, 63.0ms
Speed: 0.0ms preprocess, 63.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
 License:  B

0: 512x640 1 license, 47.3ms
Speed: 0.0ms preprocess, 47.3ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 license, 47.5ms
Speed: 0.0ms preprocess, 47.5ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
 License:  B

0: 512x640 1 license, 47.1ms
Speed: 0.0ms preprocess, 47.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
 License:  B

0: 512x640 1 license, 44.0ms
Speed: 0.0ms preprocess, 44.0ms inference, 0.0ms pos